In [ ]:
let tilelayer = L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
  attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a>'
});

let baseMap = {
  'base map': tilelayer
};

let legend = L.control({ position: 'bottom right' });

let earthquakeLayer = {
  'earthquakes': L.layerGroup(),
  'legend': legend
};

let myMap = L.map('map', {
  center: [37.7566, -119.5969],
  layers: [earthquakeLayer['earthquakes'], baseMap['base map']],
  zoom: 7
});

L.control.layers(baseMap, earthquakeLayer).addTo(myMap);

function createLegend(){
    legend.onAdd = function (map) {
    let div = L.DomUtil.create('div', 'legend');
      let colors = [
        { color: '#00FF00', label: 'Depth <= 10' },
        { color: '#FFFF00', label: '10 < Depth <= 30' },
        { color: '#FFA500', label: '30 < Depth <= 50' },
        { color: '#FF0000', label: 'Depth > 50' }
      ];
      colors.forEach(color => {
          div.innerHTML += `<i style="background: ${color.color}"></i> ${color.label}<br>`;
        });
        return div;
      };
      legend.addTo(map);};

earthquakeLayer['legend'].addTo(myMap);

const link = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson';

function getColor(depth) {
  if (depth <= 10) {
    return "#00FF00"; // Green
  } else if (depth <= 30) {
    return "#FFFF00"; // Yellow
  } else if (depth <= 50) {
    return "#FFA500"; // Orange
  } else {
    return "#FF0000"; // Red
  }};

function createMarkers(response) {
    let eFeatures = response.features;
    console.log(response);

    for (let i = 0; i < eFeatures.length; i++) {
      let eFeature = eFeatures[i];
      let eCoords = eFeature.geometry.coordinates;
      let eMag = eFeature.properties.mag;
      let eDepth = eCoords[2];
      let eRadius = eMag * 2;
      let depthColor = getColor(eDepth);

      let eMarker = L.circleMarker([eCoords[1], eCoords[0]], {
        radius: eRadius,
        fillColor: depthColor,
        fillOpacity: 0.7,
        color: "#000",
        weight: 1, 
      })
            
      eMarker.bindPopup(`Magnitude: ${eMag}<br>Depth: ${eDepth}`);
      earthquakeLayer['earthquakes'].addLayer(eMarker);
    }

  };

d3.json(link)
  .then(response => createMarkers(response));